In [47]:
import numpy as np
import scipy

In [237]:
def get_projection(q, x):
    # solve min |y-x|_2^2, s.t sum(y) = q, y >= 0
    q = np.float(q)
    n = len(x)
    min_x = np.min(x)
    x_sum = np.sum(x)
    sorted_x = np.sort(x)
    for i in range(n-1):
        lambda_2 = ((x_sum - np.sum(sorted_x[:i])) - q) / np.float(n - i)
        if lambda_2 <= min_x:
            if i == 0:
                thres = -np.inf
            else:
                thres = sorted_x[i-1]
            res = (x - lambda_2)
            res[x <= thres] = 0
            return res

In [228]:
x

array([8.36576528e-01, 2.36509688e-09, 2.45804766e-09, 4.44921945e-01,
       6.48043754e-03, 6.65211422e-09, 7.22082635e-01, 6.48681756e-01,
       3.94535662e-09, 8.09268819e-01, 2.21394370e-09, 2.39537730e-08,
       3.61426841e-02, 9.20642672e-01, 3.63456943e-09, 2.41798630e-09,
       2.45334976e-09, 1.21241241e-01, 2.54266682e-09, 4.53961229e-01])

In [229]:
get_projection(5, np.random.rand(20))

array([0.52763182, 0.        , 0.38565815, 0.0565248 , 0.28274823,
       0.        , 0.5784813 , 0.        , 0.        , 0.13740064,
       0.60825819, 0.02520689, 0.02380493, 0.12863033, 0.29770959,
       0.5333808 , 0.16338294, 0.61738011, 0.04822385, 0.58557744])

In [230]:
from cvxopt import matrix, solvers

In [231]:
Q = 2*matrix([ [2, .5], [.5, 1] ])
p = matrix([1.0, 1.0])
G = matrix([[-1.0,0.0],[0.0,-1.0]])
h = matrix([0.0,0.0])
A = matrix([1.0, 1.0], (1,2))
b = matrix(1.0)
sol=solvers.qp(Q, p, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  2e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  2e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  2e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  1e-16  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  1e-16  5e-08
Optimal solution found.


In [232]:
def solve_cvx(q, x):
    q = np.float(q)
    n = len(x)
    Q = matrix(np.eye(n), (n,n))
    p = matrix(-2 * x, (n, 1))
    G = matrix(-np.eye(n), (n, n))
    h = matrix(np.zeros(n), (n, 1))
    A = matrix(np.ones(n), (1, n))
    b = matrix(q)
    sol = solvers.qp(Q, p, G, h, A, b)
    return np.array(sol['x']).flatten()

In [251]:
tmp_x = np.random.rand(6000)
target = 200
x1 = solve_cvx(target, tmp_x)
x2 = get_projection(target, tmp_x)

     pcost       dcost       gap    pres   dres
 0: -9.4808e+02 -7.4674e+02  1e+04  1e+02  1e+00
 1: -4.7488e+02 -5.2797e+02  7e+02  5e+00  6e-02
 2: -3.1715e+02 -4.0922e+02  9e+01  1e-15  7e-16
 3: -3.4240e+02 -3.5581e+02  1e+01  1e-15  2e-16
 4: -3.5084e+02 -3.5205e+02  1e+00  1e-15  2e-16
 5: -3.5173e+02 -3.5179e+02  6e-02  1e-15  2e-16
 6: -3.5177e+02 -3.5177e+02  2e-03  1e-15  2e-16
 7: -3.5177e+02 -3.5177e+02  1e-04  1e-15  2e-16
Optimal solution found.


In [252]:
print np.sum(x1)
print np.sum(x2)

199.99999999999972
200.00000000000003


In [253]:
print np.linalg.norm(x1 - tmp_x)
print np.linalg.norm(x2 - tmp_x)

40.55861073458974
42.147728962670065
